In [1]:
BASE_PATH = '..\\data\\'
ORIGINAL_BASE_PATH = '..\\data\\original\\'
DIR_SEPARATOR = '\\' # /

ICD10_BG_4SIGN = 'ICD10_bg_4sign.csv'
ICD10_BG_3SIGN = 'ICD10_bg_3sign.csv'
ICD10_ALL_4SIGN = 'ICD10_all_4sign.csv'
ICD10_ALL_3SIGN = 'ICD10_all_3sign.csv'

LABEL_COLUMN = 'ICD10'
TEXT_COLUMN = 'Text'

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np

In [3]:
import os
from os import path

In [8]:
file_name = '{0}train-4.csv'.format(BASE_PATH)
train_df = pd.read_csv(file_name)

In [9]:
file_name = '{0}test-4.csv'.format(BASE_PATH)
test_df = pd.read_csv(file_name)

In [13]:
TEXT_COLUMN = 'Text'
LABEL_COLUMN = 'ICD10'
PRED_CLASS = 'pred_class'

In [14]:
train_df[TEXT_COLUMN] = train_df[TEXT_COLUMN].apply(lambda x:  str(x))
#dev_df[TEXT_COLUMN] = dev_df[TEXT_COLUMN].apply(lambda x:  str(x))
test_df[TEXT_COLUMN] = test_df[TEXT_COLUMN].apply(lambda x:  str(x))

train_df[PRED_CLASS] = train_df[PRED_CLASS].astype(int)
#dev_df[PRED_CLASS] = dev_df[PRED_CLASS].astype(int)
test_df[PRED_CLASS] = test_df[PRED_CLASS].astype(int)

In [15]:
train_array = train_df[TEXT_COLUMN].to_numpy()
test_array = test_df[TEXT_COLUMN].to_numpy()

In [16]:
intersection_test = np.intersect1d(test_array, train_array)

In [17]:
len(intersection_test)/len(test_array)

0.7681189506188858

In [21]:
test_df.head()

,Text,pred_class
0,сурдитас x аутозома доминант дфна10,3451
1,свърза chs eds,7479
2,вулнер,7959
3,инфекци листери,180
4,неуропатиа конгенит хипомиелинатинг вел амиели...,2902


In [18]:
!pip install nlpaug numpy matplotlib python-dotenv setuptools requests

In [19]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as nacw

In [62]:
def augment_text(text):
  aug_char = nac.RandomCharAug(action="swap", aug_char_min=1, min_char=1)
  aug_char2 = nac.RandomCharAug(action="insert", aug_char_min=1, min_char=1)
  aug_word = nacw.RandomWordAug(action="swap")
  tokens = text.split(' ')
  tokens_len = len(tokens)

  if len(text) == 1:
    augmented_text = text
  elif tokens_len == 1:
    try:      
      augmented_text = aug_char.augment(text)
    except:
      augmented_text = text
  else:    
    augmented_text = aug_word.augment(text)

  if augmented_text == text:
    augmented_text = aug_char2.augment(augmented_text)

  return augmented_text

In [63]:
augment_text('таз')

'атз'

In [64]:
augment_text('сурдитас x аутозома доминант дфна10')

'сурдитас x аутозома дфна10 доминант'

In [65]:
'сурдитас x аутозома доминант дфна10' in test_array

True

In [66]:
new_test_df = pd.DataFrame({
        TEXT_COLUMN:test_df[TEXT_COLUMN].apply(lambda x: augment_text(x) if x in test_array else x),
        LABEL_COLUMN:test_df[PRED_CLASS],
        'OLD_TEXT':test_df[TEXT_COLUMN]
        })

In [67]:
new_test_array = new_test_df[TEXT_COLUMN].to_numpy()

In [68]:
intersection_new_test = np.intersect1d(new_test_array, train_array)

In [69]:
len(intersection_new_test)/len(new_test_array)

0.021745335651451852

In [71]:
new_test_df.to_csv('{0}new-test-4.csv'.format(BASE_PATH), header=True, index=False)